In [1]:
import h2o
import pandas as pd
import numpy as np

In [2]:
h2o.init()
h2o.remove_all() 



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /tmp/tmpPdnOKN/h2o_david_started_from_python.out
JVM stderr: /tmp/tmphhycI0/h2o_david_started_from_python.err
Using ice_root: /tmp/tmp6ELYC2


Java Version: java version "1.7.0_91"
OpenJDK Runtime Environment (IcedTea 2.6.3) (7u91-2.6.3-0ubuntu0.14.04.1)
OpenJDK 64-Bit Server VM (build 24.91-b01, mixed mode)


Starting H2O JVM and connecting: ............... Connection successful!


H2O cluster uptime:,1 seconds 765 milliseconds
H2O cluster version:,3.6.0.3
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,1.72 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [3]:
df = h2o.import_file("../data/raw/train_users.csv")


Parse Progress: [##################################################] 100%


In [4]:
df = df.drop('id')

In [6]:
X = df.col_names[:-1]
del X[2]

y = df.col_names[-1]

### Deep Learning Model

In [24]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

# stopping_metric="misclassification"
dl = H2OGradientBoostingEstimator(ntrees=43, stopping_metric=5, learn_rate=0.05, 
                                  max_depth=6, balance_classes=True, nfolds=2)

In [ ]:
dl.train(X, y, training_frame=df)


gbm Model Build Progress: [####                                              ] 09%

In [13]:
print "R2:", dl.r2()

R2: 0.928891653141


### Predictions

In [14]:
test = h2o.import_file("../data/raw/test_users.csv")


Parse Progress: [##################################################] 100%


In [15]:
pred = dl.predict(test.drop('id'))

### Make CSV for submission

In [16]:
p_df = pred.as_data_frame()
p_df = pd.DataFrame(p_df)
p_df = p_df.transpose()
p_df.columns = p_df.iloc[0]
p_df = p_df.reindex(p_df.index.drop(0))
p_df.drop('predict', axis=1, inplace=True)

,AU,CA,DE,ES,FR,GB,IT,NDF,NL,PT,US,other
1,0.00141290763714149,0.004763910695875966,0.0023476981897473676,0.009755571702855244,0.014589652123061883,0.0076389808218474495,0.01225636679732363,0.5140769419560183,0.0025578444155556256,6.157228626713122E-4,0.38548625474870285,0.044498148049199035
2,1.8048763433425633E-4,6.533451667766129E-4,3.2489963802310697E-4,0.001130209853954223,0.0023390621353495413,0.0011210607867409197,0.0019077791321101942,0.9405265609408978,3.430385135566461E-4,9.033893562663203E-5,0.04537001537212576,0.00601320189050443
3,9.296011533303252E-4,0.005351550884569012,0.002258598050648964,0.007895842830869566,0.01306731449225453,0.006668862602880188,0.009673607719168286,0.7628509527334921,0.001950593901927209,5.967373758834362E-4,0.1534191235876106,0.0353372146673657
4,8.918715592719976E-4,0.003864203115820732,0.0018661843165890554,0.007534953838260957,0.012908722294508807,0.006466192188490315,0.008760362780632297,0.7831545755056414,0.001783925978246437,4.752801716836909E-4,0.13929754554193932,0.03299618270891501
5,0.0010872109045670304,0.004921342207524069,0.0019974636231272777,0.007252750415921498,0.016706557658778654,0.007835786414681514,0.011789059648128123,0.7595478086788277,0.002449837010387244,7.875730417658815E-4,0.1532572579565107,0.032367352439780325
6,0.0029200802972414305,0.010133536304545507,0.004719087298139975,0.014820943607400492,0.02846263837093351,0.0167256628155362,0.024322023321754833,0.3705514032097832,0.006431994563445248,0.0011741118028777218,0.4585141665769161,0.06122435183142584
7,0.005554961446155256,0.008199710814761577,0.0036059730699922747,0.007980783971620197,0.02144000766696545,0.014065453166307609,0.02320182865305672,0.4219648149528715,0.003852491480619421,5.17363022975639E-4,0.42913765986542124,0.06047895188925304
8,0.001039627758357847,0.005265286478538153,0.0021373978585564547,0.00722123571358158,0.01643105542826539,0.007849374263436518,0.010971994890336035,0.7562474161751916,0.002683031306574464,8.035525145827488E-4,0.1560568859394402,0.033293141673139105
9,0.001039627758357847,0.005265286478538153,0.0021373978585564547,0.00722123571358158,0.01643105542826539,0.007849374263436518,0.010971994890336035,0.7562474161751916,0.002683031306574464,8.035525145827488E-4,0.1560568859394402,0.033293141673139105
10,8.463602967734302E-4,0.00418982105654453,0.002102284088770412,0.005533625331833504,0.013114015560970938,0.005538792388552826,0.009873147423113198,0.7980875771144595,0.001339088880057792,3.2642325740503745E-4,0.12806959579746727,0.030979268804051475


In [17]:
ids = pd.DataFrame(test.as_data_frame())
ids = ids.transpose()
ids.columns = ids.iloc[0]
ids = ids.reindex(ids.index.drop(0))

In [19]:
A = []
for item in xrange(p_df.shape[0]):
    s = p_df.iloc[item]
    s = s.astype(float)
    s = s.sort_values(ascending=False)
    for i in xrange(5):
        user_id = ids.iloc[item]['id']
        dest = list(s.index[0:5])[i]
        A.append([user_id, dest])

In [20]:
sub = pd.DataFrame(A, columns = ['id', 'country'])

Write CSV file:

In [21]:
sub.to_csv('../data/submissions/sub_h2o.csv',index=False)

In [ ]:
h2o.shutdown()